In [ ]:
import sklearn
import keras
import sys
import time
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import math
import cv2

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dropout, Flatten, Dense, Bidirectional
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, Input, BatchNormalization, concatenate
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle

import skimage
from skimage import io
from skimage.transform import resize

from numpy.random import RandomState

In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re

lemmatizer = WordNetLemmatizer() 
stopwords = nltk.corpus.stopwords.words('english')

def preprocess(data):
    newData = []
    for title in data:
        title = re.sub(r'[0-9]+', '', title)
        new = " "
        for word in title.split(' '):
            
            if word not in stopwords:
                new += lemmatizer.lemmatize(word) + ' '
        newData.append(new)
        
    return newData

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
def getMaximumLen(data):
    maxL = 0
    for tweet in data:
        l = 0
        for word in tweet.split(' '):
            l += 1
        if (l>maxL):
            maxL = l    
    return maxL

In [ ]:
columns = ["Image Path", "Title", "Category ID", "Category"]
data = pd.read_csv('train.csv', encoding = "ISO-8859-1", header=None,
                   usecols=[1,3,6,5], names=columns)

In [ ]:
image = data['Image Path']
x = data['Title'].str.lower()
y = to_categorical(LabelEncoder().fit_transform(data['Category']), num_classes=30)
x = preprocess(x)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfs = tfidf.fit_transform(x)
print('Shape of TF-IDF matrix: ', tfs.T.shape)

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(tfs, y,  train_size=0.8, random_state=1)

In [ ]:
from keras import *
from keras.layers import Dense
from keras.utils import to_categorical
import functools

tf_input = Input(shape=(tfs.shape[1],), dtype='float32')
x= Dense(512,activation='relu')(tf_input)
x= Dropout(0.5)(x)
x= Dense(128,activation='relu')(x)
x= Dropout(0.5)(x)
x= Dense(64,activation='relu')(x)
x= Dropout(0.5)(x)
x= Dense(30,activation='softmax')(x)
bagOfWords = Model(tf_input, x)


top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

bagOfWords.compile(keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy', top3_acc, f1_m])



bagOfWords.fit(xtrain, ytrain, validation_data=(xval, yval),epochs=50, batch_size=128,
               callbacks=[EarlyStopping(monitor='val_loss', patience=10),
                           ModelCheckpoint(filepath='bow.h5', monitor='val_loss', save_best_only=True))